# Creating a Simple Agent to Play Super Mario Using PPO RL Algorithm Applied on a CNN 

## Importing Libraries

In [ ]:
#Use Older Version of Pip, Wheel and Setuptools for the project

import gym_super_mario_bros as gsbm #Use Version 7.3.0
from nes_py.wrappers import JoypadSpace
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT
import gym #Use OpenAI Gym V0.21

In [ ]:
from gym.wrappers import GrayScaleObservation
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv

In [ ]:
import os
from stable_baselines3 import PPO #Use Older Version of Stable-Baselines3 (v1.5 or v1.6)
from stable_baselines3.common.callbacks import BaseCallback #Use Older Version of Stable-Baselines3 (v1.5 or v1.6)

## Applying Preprocessing to Gym Environment Using Imported Wrappers

In [ ]:
env = gsbm.make('SuperMarioBros-v0') #Making a custom Mario gym environment
env = JoypadSpace(env, SIMPLE_MOVEMENT) #Using to reduce action space
env = GrayScaleObservation(env, keep_dim=True) #Wrapping to convert RGB --> GrayScale
env = DummyVecEnv([lambda: env]) #Wrapping in dummy env
env = VecFrameStack(env, 4, channels_order='last') #Stacking frames

## Defining a Callback Function to Save Model Every N TimeStamp During Training

In [ ]:
class TrainAndLoggingCallback(BaseCallback):
    
    def __init__ (self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path
        
    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)
            
    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self .model.save(model_path)
        return True

In [ ]:
#defining folders for storing models
CHECKPOINT_DIR = './RL_MODEL/' 
#defining folders for storing logs
LOG_DIR = './logs/' 

In [ ]:
#setting up the callback function
callback = TrainAndLoggingCallback(check_freq=100000, save_path = CHECKPOINT_DIR) 

## Setting Up and Training the Model

In [ ]:
#setting up the model
model = PPO('CnnPolicy', env, verbose=1, tensorboard_log = LOG_DIR, learning_rate = 0.000001, n_steps = 512 ) 

In [ ]:
#setting up the learning process of the model
model.learn(total_timesteps = 10000000, callback=callback) 

## Making Predictions Using the Model (Playing the Game)

In [ ]:
#loading the model
model = PPO.load('./RL_MODEL/best_model_5100000') 

In [ ]:
#initializing the environment
state = env.reset() 

In [ ]:
#Rendering the environment with actions being predicted by the agent, essentially making it play the game
while True:
    action, _ = model.predict(state)
    observation, reward, terminated, truncated, info = env.step(action)
    env.render()

In [ ]:
env.close() #used to close the game post interrupting above loop